In [80]:
import torchvision
from torchvision import transforms
import torch
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ExponentialLR, CosineAnnealingLR
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
import pickle
from BaseModel import CustomCNN
import torch.nn as nn
import torch
import torch.optim as optim
import json
from torch.utils.data import DataLoader,TensorDataset
import numpy as np

In [83]:
with open('D:\DPL302m\project\Abnormal_Behavior\Model\param.json', 'r') as json_file:
    data = json.load(json_file)

for key, value in data.items():
    globals()[key] = value

In [84]:
def LoadData(name=""):
    data = np.load(name+".npz")
    images = data['images']
    labels = data['labels']

    images = torch.from_numpy(images)
    labels = torch.from_numpy(labels)

    dataset = TensorDataset(images, labels)
    return dataset


In [85]:
#Config

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('DEVICE:',DEVICE)
model = CustomCNN(num_classes=2)
criterion = nn.CrossEntropyLoss()
model.to(DEVICE)
criterion.to(DEVICE)
batch_size = BATCH_SIZE

optimizer = optim.Adam(model.parameters(), lr=float(LEARNING_RATE))
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)

TRAINLOADER = DataLoader(LoadData('Data_train'), batch_size=batch_size, shuffle=True)
TESTLOADER = DataLoader(LoadData('Data_test'), batch_size=batch_size, shuffle=False)

device(type='cpu')

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in TRAINLOADER:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).long()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss_train = criterion(outputs,labels)
        loss_train.backward()
        optimizer.step()
    if scheduler is not None:
        scheduler.step()
    
    model.eval() 
    with torch.no_grad():
        for data in TESTLOADER:
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).long()
            outputs = model(inputs)
            loss_test = criterion(outputs, labels)
            
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss Train: {loss_train.item():.4f} Loss Test: {loss_test.item():.4f}")

In [86]:

# Initialize variables for early stopping and best model
early_stopping_counter = 0
best_test_loss = float('inf')
best_model_weights = None
loss_train = []
loss_test = []
f1_train = []
f1_test = []
for epoch in range(EPOCHS):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    predictions_train = []
    true_labels_train = []

    for i, data in tqdm(enumerate(TRAINLOADER), desc='train'):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        predictions_train.extend(predicted.tolist())
        true_labels_train.extend(labels.tolist())

    if scheduler is not None:
        scheduler.step()  # Update learning rate with scheduler

    train_loss = running_loss / len(TRAINLOADER)
    train_f1 = f1_score(true_labels_train, predictions_train, average='weighted')

    loss_train.append(train_loss)
    f1_train.append(train_f1)

    # Evaluation on the test set
    model.eval()  # Set the model to evaluation mode
    test_loss_val = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for data in TESTLOADER:
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss_val += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    test_loss_val /= len(TESTLOADER)
    test_f1_val = f1_score(true_labels, predictions, average='weighted')

    loss_test.append(test_loss_val)
    f1_test.append(test_f1_val)
    print(f'Epoch [{epoch + 1}/{EPOCHS}]  - Train Loss: {train_loss:.4f} - Train F1: {train_f1:.4f} - Test Loss: {test_loss_val:.4f} - Test F1: {test_f1_val:.4f}')

    # Check if the test loss has improved
    if test_loss_val < best_test_loss:
        best_test_loss = test_loss_val
        early_stopping_counter = 0
        # Save the weights of the best model
        best_model_weights = model.state_dict()
    else:
        early_stopping_counter += 1

    # Check for early stopping
    if early_stopping_counter >= 5:
        print("Early stopping triggered. No improvement for 5 consecutive epochs.")
        break

print('Finished Training')

# Load the best model weights before saving
if best_model_weights is not None:
    model.load_state_dict(best_model_weights)

# Save the weights of the best model
torch.save(model.state_dict(), 'best_model_weights.pth')
print('Best model weights saved as best_model_weights.pth')


train: 1it [00:00,  1.56it/s]


Epoch [1/10]  - Train Loss: 0.7028 - Train F1: 0.4205 - Test Loss: 0.7071 - Test F1: 0.3333


train: 1it [00:00,  1.68it/s]


Epoch [2/10]  - Train Loss: 0.7155 - Train F1: 0.6496 - Test Loss: 0.7973 - Test F1: 0.3333


train: 1it [00:00,  1.74it/s]


...

Epoch [8/10]  - Train Loss: 0.1023 - Train F1: 1.0000 - Test Loss: 0.6360 - Test F1: 0.3333


train: 1it [00:00,  1.89it/s]


Epoch [9/10]  - Train Loss: 0.0777 - Train F1: 1.0000 - Test Loss: 0.6030 - Test F1: 0.6190


train: 1it [00:00,  1.87it/s]

Epoch [10/10]  - Train Loss: 0.0512 - Train F1: 1.0000 - Test Loss: 0.5544 - Test F1: 0.8730
Finished Training
Best model weights saved as best_model_weights.pth
